<a href="https://colab.research.google.com/github/imyoungchae/Tensorflow_study/blob/main/tensorflow6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN(Convolution Neural network)를 활용한 image Classification

# Import

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

# Load Dataset

In [ ]:
dataset_name = 'cats_vs_dogs'

# 처음 80%의 데이터만 사용
train_dataset = tfds.load(name=dataset_name, split='train[:80%]')

# 최근 20%의 데이터만 사용
valid_dataset = tfds.load(name=dataset_name, split='train[80%:]')

시험에서 요구하는 전처리 요구 조건
1. 이미지 정규화 (Nomalization)
2. 이미지 사이즈 맞추기 (244x244)
3. image(x),label(y)를 분할

In [ ]:
def preprocess(data):
    # x, y 데이터를 정의합니다.
    x = data['image']
    y = data['label']
    # image 정규화(Normalization)
    x = x / 255
    # 사이즈를 (224, 224)로 변환합니다.
    x = tf.image.resize(x, size=(224, 224))
    # x, y  데이터를 return 합니다.
    return x, y

만든 전처리 함수(preprocessing)을 dataset에 mapping하고 batch_size 지정

In [ ]:
batch_size=32

In [ ]:
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

# Sequential

이제 Modeling을 할 차례
1. input_shape는 244x244 컬러사진인 (244,244,3)으로 지정
2. transfer learning 기법을 통해 VGG16 모델을 활용한 전이학습 모델 완성
3. 출력층은 class 갯수 2개의 뉴런 요구

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
transfer_model.trainable=False # Freeze: 윗단계에서 학습이 일어나지 않도록

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
model = Sequential([
    transfer_model,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax'), # 개, 고양이 분류니까 Dense 2
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 27,626,178
Trainable params: 12,911,490
No

# Compile

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

# ModelCheckpoint

In [ ]:
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

# Fit

In [ ]:
model.fit(train_data,
          validation_data=(valid_data),
          epochs=10,
          callbacks=[checkpoint],
          )

Epoch 1/10
 60/582 [==>...........................] - ETA: 2:36 - loss: 0.1296 - acc: 0.9505

# Load Weights

In [ ]:
# checkpoint 를 저장한 파일명을 입력
model.load_weights(checkpoint_path)